<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_CD_2024/blob/main/IMT_CD_P4A_2024S2_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/Rogerio-mack/IMT_CD_2024/blob/main/maua_logo.png?raw=true" width=300, align="right">
<br>

In [ ]:
#@markdown imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif

from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score



# Case: Blood transfusion

Considere os dados abaixo sobre indivíduos potenciais doadores de sangue.

In [ ]:
df = pd.read_csv("https://github.com/Rogerio-mack/IA_2024S2/raw/refs/heads/main/data/blood_donate.csv")
df.head()

,Recency,Frequency,Monetary,Time,Class
0,2,50,12500,98,donated
1,0,13,3250,28,donated
2,1,16,4000,35,donated
3,2,20,5000,45,donated
4,1,24,6000,77,not donated


# Ex1. Classification

Avalie com o `GridSearchCV()` os seguintes estimadores para Classe dos indivíduos:

<br>

> **Estimadores**

* `DecisionTreeClassifier(random_state=42)`, variando a profundidade da árvore de 5-10 e os critérios de ganho de informação 'gini','entropy','log_loss'.

* `SVC(max_iter=1000,random_state=42)`, variando todos os tipos de `kernel` disponíveis, **exceto** o `precomputed` que requer a criação de uma função sua para `kernel` .

> Em ambos os casos não esqueça de empregar nos estimadores `random_state=42` para a garantia da reprodutibilidade dos resultados. Você pode optar por empregar um laço de programa para avaliar os dois estimadores ou fazer avaliações separadas. Fica a seu critério. Na SVC ignore eventuais warnings de não convergência.

<br>


> **Conjunto de Treinamento e Teste, CV**

* Empregue 25% de dados de teste, **não estratificados**. Não esqueça de empregar `random_state=42` para a garantia da reprodutibilidade dos resultados. Empregue 5 partições de CV.

> Para a garantia da reprodutibilidade dos resultados **não empregue outros parâmetros que não foram solicitados.**

<br>


> **Pré-Processamento dos dados**

* Aplique a normalização `StandardScaler()`.


In [ ]:
#
# Separa os conjuntos de Treinamento e Teste
#
from sklearn.model_selection import train_test_split

X = df.drop(['Class'], axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((561, 4), (561,), (187, 4), (187,))

In [ ]:
#
# Normaliza os dados
#
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#@markdown sanity check, must be True
X_test_scaled.sum().sum() == -93.22138228732187

True

In [ ]:
#
# Avalia com o GridSearchCV() os diferentes estimadores e seus parâmetros
#
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Q11 = []

base_estimators = {
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
    'SVC': SVC(max_iter=1000,random_state=42)
}

parm_grid = {
    'DecisionTreeClassifier': {'max_depth': range(5, 11),'criterion': ['gini','entropy','log_loss']},
    'SVC': {'kernel': ['linear', 'rbf', 'sigmoid', 'poly']}
}

for name, base_estimator in base_estimators.items():
    print(f"Training {name}...")
    clf = GridSearchCV(base_estimator, parm_grid[name], cv=5)
    clf.fit(X_train, y_train)

    print(f'best estimator: {clf.best_estimator_} with score {clf.best_score_:.3f}')
    y_pred = clf.predict(X_test)

    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test, y_pred))
    print(f'accuracy score in X_test: {clf.score(X_test, y_test):.3f}')

    Q11.append((name, clf.best_estimator_, clf.score(X_test, y_test)))


Training DecisionTreeClassifier...
best estimator: DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42) with score 0.775
[[ 20  28]
 [ 23 116]]
              precision    recall  f1-score   support

     donated       0.47      0.42      0.44        48
 not donated       0.81      0.83      0.82       139

    accuracy                           0.73       187
   macro avg       0.64      0.63      0.63       187
weighted avg       0.72      0.73      0.72       187

accuracy score in X_test: 0.727
Training SVC...


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider

best estimator: SVC(max_iter=1000, random_state=42) with score 0.772
[[  1  47]
 [  0 139]]
              precision    recall  f1-score   support

     donated       1.00      0.02      0.04        48
 not donated       0.75      1.00      0.86       139

    accuracy                           0.75       187
   macro avg       0.87      0.51      0.45       187
weighted avg       0.81      0.75      0.65       187

accuracy score in X_test: 0.749


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


## **Q1.1.A. Qual o melhor modelo de Árvore de Decisão obtido e sua acuracidade no conjunto de Teste**?




## **Q1.1.B. Qual o melhor modelo de Máquina Vetorial de Suporte (SVC) obtido e sua acuracidade no conjunto de Teste**?

## **Q1.2. Ao final qual o melhor estimador a ser empregado e por quê?**

Atenção: Avalie outras métricas do `Classification Report`!

In [ ]:
Q12 = 'DecisionTreeClassifier, o SVC não identifica a quase totalidade dos casos de "donated"'

## **Q1.3. (PCA) Qual o resultado (acurácia) do melhor modelo selecionado no exercício anterior com o uso de Componentes Principais que correspondam a 80% da variância dos dados?**

**Nota:**

1. Aplique o PCA diretamente no mesmos dados de treinamento e teste já selecionados. Lembre-se de aplicar corretamente as tranformações `fit_transform()` e `transform()`.



## **Errata**: aplique o PCA em X_train e X_test, sem normalizar

In [ ]:
#
# Aplica o PCA
#
from sklearn.decomposition import PCA

pca = PCA(0.8)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print(f"Número de componentes principais retidos: {pca.n_components_} com {pca.explained_variance_ratio_} da variância dos dados.")


Número de componentes principais retidos: 1 com [0.99981604] da variância dos dados.


In [ ]:
#@markdown sanity check, must be True
X_test_pca.sum().sum() == -37088.5381011182

True

In [ ]:
#
# Aplica o estimador de classificação aos novos dados redimensionados
#
clf = DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42)
clf.fit(X_train_pca, y_train)

y_pred = clf.predict(X_test_pca)

print(classification_report(y_test, y_pred))
print(f'accuracy score in X_test: {clf.score(X_test_pca, y_test):.3f}')



              precision    recall  f1-score   support

     donated       0.80      0.08      0.15        48
 not donated       0.76      0.99      0.86       139

    accuracy                           0.76       187
   macro avg       0.78      0.54      0.51       187
weighted avg       0.77      0.76      0.68       187

accuracy score in X_test: 0.759


In [ ]:
Q13 = (clf.score(X_test_pca, y_test), pca.n_components_, pca.explained_variance_ratio_)

# Ex2. Clustering

Aplique a Clusterização Kmeans e Hierárquica aos dados preditores (isto é, exceto o atributo Class) aplicando um  PCA (min 80% da variância dos dados), para a formação de 2 grupos (clusters) de indivíduos. Note, **o PCA, agora é aplicado a todos os dados, `X`**, e não somente aos dados de treinamento. Portanto, não aplique o estimador anterior!

Para a clusterização Hierárquica empregue o **linkage de distância máxima**, e no Kmeans, não deixe de empregar o `random_state=42` para reprodutibilidade dos resultados.

Responda as questões sobre os grupos, e em seguida verifique a **Acurácia de Clusterização** (isto é, a coincidência com os grupos formados e suas classes).



In [ ]:
from sklearn.preprocessing import StandardScaler

X = df.drop(['Class'], axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
#
# Aplique o PCA aos dados normalizados
#

pca = PCA(0.8)
X_pca = pca.fit_transform(X_scaled)


In [ ]:
#@markdown sanity check, must be True
X_pca.sum().sum() == 1.4210854715202004e-14

True

In [ ]:
#
# Faça as Clusterizações Kmeans e Hierárquica
#
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_pca)
print(f'Kmeans silhouette_score average: {silhouette_score(X_pca, kmeans.labels_):.3f}, com {sum(kmeans.labels_==0)} e {sum(kmeans.labels_==1)} elementos em cada grupo')

hclust = AgglomerativeClustering(n_clusters=2,linkage='complete')
hclust.fit(X_pca)
print(f'Hclust silhouette_score average: {silhouette_score(X_pca, hclust.labels_):.3f}, com {sum(hclust.labels_==0)} e {sum(hclust.labels_==1)} elementos em cada grupo')




Kmeans silhouette_score average: 0.484, com 141 e 607 elementos em cada grupo
Hclust silhouette_score average: 0.759, com 740 e 8 elementos em cada grupo


## **Q2.1.A. Qual a silhueta média obtida na clusterização Hierárquica e o número de indivíduos de cada grupo?**



## **Q2.1.B. Qual a silhueta média obtida na clusterização Kmeans e o número de indivíduos de cada grupo?**

In [ ]:
Q21 = {'A':(silhouette_score(X_pca, hclust.labels_),sum(hclust.labels_==0),sum(hclust.labels_==1)),
       'B':(silhouette_score(X_pca, kmeans.labels_),sum(kmeans.labels_==0),sum(kmeans.labels_==1))}

## **Q2.2.A. Qual cluster hierárquico apresenta maior renda (`Monetary`) e qual esse valor médio?**



## **Q2.2.B. Qual cluster kmeans estão associados os indivíduos 0, 5, 6 e 8?**

In [ ]:
df['hclust'] = hclust.labels_
df['kmeans'] = kmeans.labels_

In [ ]:
print(df['hclust'].value_counts())
print(df['kmeans'].value_counts())
print(df['Class'].value_counts())

hclust
0    740
1      8
Name: count, dtype: int64
kmeans
1    607
0    141
Name: count, dtype: int64
Class
not donated    570
donated        178
Name: count, dtype: int64


In [ ]:
display(df.drop(columns=['Class','hclust']).groupby('kmeans').mean())
display(df.drop(columns=['Class','kmeans']).groupby('hclust').mean())
display(df.drop(columns=['kmeans','hclust']).groupby('Class').mean())

,Recency,Frequency,Monetary,Time
kmeans,,,,
0,7.255319,14.347518,3586.879433,66.184397
1,10.029654,3.462932,865.733114,26.871499


,Recency,Frequency,Monetary,Time
hclust,,,,
0,9.552703,5.12973,1282.432432,33.637838
1,5.250000,41.12500,10281.250000,93.875000


,Recency,Frequency,Monetary,Time
Class,,,,
donated,5.455056,7.797753,1949.438202,32.719101
not donated,10.771930,4.801754,1200.438596,34.770175


In [ ]:
df.iloc[[0,5,6,8],:]

,Recency,Frequency,Monetary,Time,Class,hclust,kmeans
0,2,50,12500,98,donated,1,0
5,4,4,1000,4,not donated,0,1
6,2,7,1750,14,donated,0,1
8,2,9,2250,22,donated,0,1


In [ ]:
Q22 = {'A':(df.drop(columns=['Class','kmeans']).groupby('hclust').Monetary.mean().argmax(),
            df.drop(columns=['Class','kmeans']).groupby('hclust').Monetary.mean()),
       'B':df.iloc[[0,5,6,8],6]}

## **Q2.3.A. Qual a Acurácia de Clusterização para os clusters kmeans com relação a `Class`?**



## **Q2.3.B. Qual a Acurácia de Clusterização para os clusters hirárquicos com relação a `Class`?**

In [ ]:
df['kmeans_Class'] = df['kmeans'].map({0:'not donated', 1:'donated'})
df['hclust_Class'] = df['hclust'].map({0:'not donated', 1:'donated'})

In [ ]:
confusion_matrix(df.Class, df['kmeans_Class']), confusion_matrix(df.Class, df['hclust_Class'])

(array([[135,  43],
        [472,  98]]),
 array([[  6, 172],
        [  2, 568]]))

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(df.Class, df['kmeans_Class'])
print(f"Acurácia de Clusterização (kmeans): {accuracy:.3f}")

accuracy = accuracy_score(df.Class, df['hclust_Class'])
print(f"Acurácia de Clusterização (hclust): {accuracy:.3f}")

Acurácia de Clusterização (kmeans): 0.311
Acurácia de Clusterização (hclust): 0.767


In [ ]:
Q23 = {'A': ('kmeans', accuracy_score(df.Class, df['kmeans_Class'])),
       'B': ('hclust', accuracy_score(df.Class, df['hclust_Class']))}

# Gabarito

In [ ]:
print(f'Q1.1.A. Qual o melhor modelo de Árvore de Decisão obtido e sua acuracidade no conjunto de Teste?')
print(Q11[0])
print()
print(f'Q1.1.B. Qual o melhor modelo de Máquina Vetorial de Suporte (SVC) obtido e sua acuracidade no conjunto de Teste?')
print(Q11[1])
print()
print(f'Q1.2. Ao final qual o melhor estimador a ser empregado e por quê?')
print(Q12)
print()
print(f'Q1.3. (PCA) Qual o resultado (acurácia) do melhor modelo selecionado no exercício anterior com o uso de Componentes Principais que correspondam a 80% da variância dos dados?')
print(Q13)
print()
print(f'Q2.1.A. Qual a silhueta média obtida na clusterização Hierárquica e o número de indivíduos de cada grupo?')
print(Q21['A'])
print()
print(f'Q2.1.B. Qual a silhueta média obtida na clusterização Kmeans e o número de indivíduos de cada grupo?')
print(Q21['B'])
print()
print(f'Q2.2.A. Qual cluster hirárquico apresenta maior renda (`Monetary`) e qual esse valor médio?')
print(Q22['A'])
print()
print(f'Q2.2.B. Qual cluster kmeans estão associados os indivíduos 0, 5, 6 e 8?')
print(Q22['B'])
print()
print(f'Q2.3.A. Qual a Acurácia de Clusterização para os clusters kmeans com relação a `Class`?')
print(Q23['A'])
print()
print(f'Q2.3.B. Qual a Acurácia de Clusterização para os clusters hirárquicos com relação a `Class`?')
print(Q23['B'])


Q1.1.A. Qual o melhor modelo de Árvore de Decisão obtido e sua acuracidade no conjunto de Teste?
('DecisionTreeClassifier', DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42), 0.7272727272727273)

Q1.1.B. Qual o melhor modelo de Máquina Vetorial de Suporte (SVC) obtido e sua acuracidade no conjunto de Teste?
('SVC', SVC(max_iter=1000, random_state=42), 0.7486631016042781)

Q1.2. Ao final qual o melhor estimador a ser empregado e por quê?
DecisionTreeClassifier, o SVC não identifica a quase totalidade dos casos de "donated"

Q1.3. (PCA) Qual o resultado (acurácia) do melhor modelo selecionado no exercício anterior com o uso de Componentes Principais que correspondam a 80% da variância dos dados?
(0.7593582887700535, 1, array([0.99981604]))

Q2.1.A. Qual a silhueta média obtida na clusterização Hierárquica e o número de indivíduos de cada grupo?
(0.7585881567810211, 740, 8)

Q2.1.B. Qual a silhueta média obtida na clusterização Kmeans e o número de indivíduos de ca